In [ ]:
#PigLab6NoteBook

! pip install pandas
! pip install openpyxl
!pip install rdflib
import pandas as pd
import rdflib
import hashlib
import numpy as np
import urllib.parse
from rdflib import Literal, Namespace, RDF, URIRef, Graph, RDFS, OWL
from rdflib.namespace import FOAF, XSD
from rdflib.plugins.sparql import prepareQuery
from pyspark.sql.functions import when, col, lit

In [ ]:
# Define the file paths
files = ['data/PigData/Lab6/Copia de Labopat_mapa diagnostico_2021-2023.xlsx']

# Load the data into Pandas dataframes
dfs = []
for file in files:
    df = pd.read_excel(file, engine='openpyxl')
    dfs.append(df)

df = dfs[0]


In [ ]:
df = df.drop(['Unnamed: 0'], axis=1)


In [ ]:
df

In [ ]:
#Rename Colums

df = df.rename(columns={
    'ID': 'SampleNumber',
    'Provicincia': 'Province',
    'Fecha': 'Date',
    'Patógeno': 'Pathogen',
    'Resultado': 'Result',
    'Muestra': 'SampleType',
    'Edad/Granja': 'Age', #Age of animal
})

# Set DiagnosticTest to "PCR" Country to "Spain", Breed to "Meat", and LabReference to "1".
df['DiagnosticTest'] = 'PCR'
df['Country'] = 'Spain'
df['Breed'] = 'Meat'
df["Lab_Reference"] = "6"






In [ ]:
#Annotation Mapping

# Add Result mapping for NEG = 0 and POS =1
resultMapping = {
    "Positivo": "1",
    "positivo": "1",
    "Negativo": "0",
    "Neg.": "0",
    "Neg": "0",
    "neg.": "0",
    "negativo": "0",
}
df["Result"] = df["Result"].map(resultMapping).fillna("Missing")

PathogenMapping = {

    "Lawsonia intracellularis": "Lawsonia intracellularis",
    "L. intracellularis": "Lawsonia intracellularis",
    "L.intracelullaris": "Lawsonia intracellularis",
    "M. hyopneumoniae": "Mycoplasma hyopneumoniae",
    "Myc. hyopneumoniae": "Mycoplasma hyopneumoniae",
    "Mycoplasma hyopneumoniae antistoffen" : "Mycoplasma hyopneumoniae",
    "Mycoplasma hyopneumoniae" : "Mycoplasma hyopneumoniae",
    "M. hyopneumoniae" : "Mycoplasma hyopneumoniae",
    "SIV" : "Swine influenza virus",
    "Influenza A" : "Swine influenza virus",
    "SIV" : "Swine influenza virus",
    "LI": "Lawsonia intracellularis",
    "MH": "Mycoplasma hyopneumoniae",
  
}

df["Pathogen"] = df["Pathogen"].map(PathogenMapping).fillna("Missing")

# SampleTypeMapping for the additional sample types
SampleTypeMapping = {
    'Pulmón': 'Tissue',
    'Intestino': 'Tissue',           # Added based on the new dataset
    'Heces': 'Feaces',             
    'Fluido Oral': 'Fluid',         
    'Hisopo Rectal': 'Swab',         
    'Hisopo Nasal': 'Swab',         
    'Hisopo Laríngeo': 'Swab',      
    'Raspados amígdalas': 'Tissue',  
    'Lechón': 'Mixed',               
    'Calzas': 'other',               
    'Raspado bronquial': 'Tissue', 
    'Hisopo pulmonar': 'Swab',       
    'Lavado Bronquial': 'other',     
    'Raspado laríngeo': 'Tissue',    
    'Exudado nasal': 'other',       
    'Hisopo Traqueal': 'Swab',      
    'Raspado Traqueal': 'Tissue', 
}

# Map the SampleType column using the defined mapping
df["SampleType"] = df["SampleType"].map(SampleTypeMapping).fillna("Missing")


# Province mapping
ProvinceMapping = {
    'Castellón': 'Castellon',
    'Huesca': 'Huesca',
    'Zaragoza': 'Zaragoza',
    'Barcelona': 'Barcelona',
    'Albacete': 'Albacete',
    'Badajoz': 'Badajoz',
    'Sevilla': 'Sevilla',
    'Segovia': 'Segovia',
    'Ávila': 'Avila',
    'Murcia': 'Murcia',
    'Burgos': 'Burgos',
    'Pontevedra': 'Pontevedra',
    'Badajoz': 'Badajoz',
    'Sevilla': 'Seville',            # Sevilla is commonly known as Seville in English
    'Valladolid': 'Valladolid',
    'Toledo': 'Toledo',
    'Asturias': 'Asturias',
    'Granada': 'Granada',
    'Lugo': 'Lugo',
    'Salamanca': 'Salamanca',
    'Zaragoza': 'Zaragoza',
    'Huelva': 'Huelva',
    'Zamora': 'Zamora',
    'Málaga': 'Malaga',              # Málaga is commonly known as Malaga in English
    'Vitoria': 'Vitoria',    # Vitoria is also known as Vitoria-Gasteiz
    'Valencia': 'Valencia',
    'A Coruña': 'La Coruna',         # A Coruña is often translated to La Coruna
    'Álava': 'Alava',
    'Almería': 'Almeria',            # Almería is commonly known as Almeria in English
    'Cuenca': 'Cuenca',
    'Ourense': 'Orense',             # Ourense is often translated to Orense
    'León': 'Leon',                  # León is commonly known as Leon in English
    'Jaén': 'Jaen',                  # Jaén is commonly known as Jaen in English
    'Madrid': 'Madrid',
    'Barcelona': 'Barcelona',
    'Ciudad Real': 'Ciudad Real',
    'Teruel': 'Teruel'
}


df["Province"] = df["Province"].map(ProvinceMapping).fillna("Missing")




In [ ]:
df

In [ ]:
# PigAge mapping
PigStageMapping = {

    'Destetados': 'Weaning',
    'Cebo': 'Finishing',
    'ND': 'Missing',
    'Cebo ': 'Finishing',
    'Cerdas': 'Sow',
    'Cebo': 'Finishing',
    'Lechones 3 meses': 'Finishing',
    '4 meses': 'Finishing',
    'Cebo ': 'Finishing',
    '-': 'Missing',
    'Lechones': 'Nursery',
    '50 días': 'Weaning',
    '1 mes': 'Weaning',
    '1-9 semanas': 'Nursery',
    '6 semanas': 'Nursery',
    '10 semanas': 'Nursery',
    'Nulíparas': 'Gilt',
    'Recría': 'Sow',
    '45 días': 'Weaning',
    '3 meses': 'Finishing',
    'Reposición': 'Sow',
    'Destete': 'Weaning',
    'Transición y Cebo': 'Finishing',
    'Primerizas': 'Gilt',
    'Cebo 100kg': 'Finishing',
    '6 meses': 'Finishing',
    'Transición  ': 'Nursery',
    '9 semanas': 'Nursery',
    '7 días': 'Suckling',
    'Cebo 130 días': 'Finishing',
    'Cerdas gestación': 'Sow',
    '8 semanas': 'Nursery',
    '80 días': 'Finishing',
    '180 días': 'Finishing',
    '10 meses': 'Finishing',
    '12 semanas': 'Nursery',
    '100 kg': 'Finishing',
    'Final cebo': 'Finishing',
    'Precebo': 'Finishing',
    '90 días': 'Finishing',
    '140 días': 'Finishing',
    '3-9 semanas': 'Nursery',
    '100 días': 'Finishing',
    'Primales': 'Gilt',
    '5 meses': 'Finishing',
    '110 días': 'Finishing',
    '18 días': 'Suckling',
    '35 Kg': 'Finishing',
    '2 meses': 'Weaning',
    '3,5 meses': 'Finishing',
    '5 semanas': 'Nursery',
    '50-80 días': 'Finishing',
    'Cebo 5 meses': 'Finishing',
    '7 semanas': 'Nursery',
    'Cerdas recría': 'Sow',
    '17 semanas': 'Nursery',
    '8-9 meses': 'Finishing',
    'Cebo 70 kg': 'Finishing',
    '26 semanas': 'Finishing',
    '3 semanas': 'Suckling',
    '14 semanas': 'Nursery',
    '25 semanas': 'Finishing',
    '60 días': 'Weaning',
    'Lactación': 'Suckling',
    '75 días': 'Finishing',
    'Transición': 'Nursery',
    '2,5 meses': 'Finishing',
    'Verraco': 'Boar',
    '35 días': 'Suckling',
    '30 días': 'Suckling',
    '16 semanas': 'Nursery',
    '40 días': 'Weaning',
    '70 días': 'Finishing',
    'Cebo 30 kg': 'Finishing',
    'Cerdas 10 meses': 'Sow',
    'Post-destete': 'Weaning',
    '9 meses': 'Finishing',
    'Cerdas Reposición': 'Sow',
    '28 semanas': 'Finishing',
    '8 meses': 'Finishing',
    '18 semanas': 'Nursery',
    '120 días': 'Finishing',
    'Cebo 120 días': 'Finishing',
    '49 días': 'Weaning',
    '11 semanas': 'Nursery',
    'Prímalas': 'Gilt',
    '24 semanas': 'Finishing',
    '21 días': 'Suckling',
    '1-3 años': 'Finishing',
    'Cebo 55 kg': 'Finishing',
    '3 días': 'Suckling',
    '25 kg': 'Finishing',
    'Cebo 14 semanas': 'Finishing',
    '90 Kg': 'Finishing',
    '80 Kg': 'Finishing',
    '54 días': 'Weaning',
    '90 kg': 'Finishing',
    '30 kg': 'Finishing',
    '20 kg': 'Finishing',
    '160 días': 'Finishing',
    '14-16 semanas': 'Nursery',
    'Precebo 15 kg': 'Finishing',
    '19 semanas': 'Nursery',
    '4 semanas': 'Suckling',
    '33 días': 'Weaning',
    'Gestación': 'Sow',
    '13 semanas': 'Nursery',
    '3-4 meses': 'Finishing',
    '5-7 semanas': 'Nursery',
    '50 kg': 'Finishing',
    '4-9 semanas': 'Nursery',
    '3-4 semanas': 'Suckling',
    'Ceno': 'Other',
    '25-30 kg': 'Finishing',
    'Entrada cebo': 'Finishing',
    '15 semanas': 'Nursery',
    '10-13 semanas': 'Nursery',
    '20 días': 'Suckling',
    'Cebo 90 kg': 'Finishing',
    'Cebo 80 kg': 'Finishing',
    'Cerdas lactación': 'Sow',
    'Cebo 40 kg': 'Finishing',
    '85 días': 'Finishing',
    'Final transición': 'Finishing',
    '20 semanas': 'Nursery',
    '22 semanas': 'Nursery',
    '2 a 7 meses': 'Finishing',
    '3 a 5 meses': 'Finishing',
    '42 días': 'Weaning',
    '2 años': 'Finishing',
    '150 días': 'Finishing'
  
}

# Map the 'Edad' column using the English translation mapping
df['PigStage'] = df['Age'].map(PigStageMapping).fillna("Finishing")
df = df.drop(['Age'], axis=1)



In [ ]:
df

In [ ]:
# Save to desired path
df.to_csv('output/PigLab6.csv', index=False)

In [ ]:
   barometer = df

In [ ]:
barometer

### Step 02: Create an RDF graph and namespaces.

In [ ]:
g = rdflib.Graph()
xsd = Namespace('http://www.w3.org/2001/XMLSchema#')
g.bind('xsd', xsd)

# Define your custom namespace for your ontology's properties
LHO = Namespace("http://www.purl.org/decide/LiveStockHealthOnto/LHO#")
g.bind('LHO', LHO)

SKOS = Namespace("http://www.w3.org/2004/02/skos/core#")
g.bind('skos', SKOS)

decide=  Namespace("http://www.purl.org/decide#")
g.bind('decide', decide)
ncit = Namespace("http://purl.obolibrary.org/obo/NCIT_C25464")
g.bind('ncit', ncit)
agrovoc = Namespace ("http://aims.fao.org/aos/agrovoc")
g.bind('agrovoc', agrovoc)

###  Step 03: Iterate over the Panda DataFrame and map to ontology properties:

In [ ]:
#For Pig Data frame RDF conversion
for index, row in barometer.iterrows():
    SampleNumber = row["SampleNumber"]
    Date = row["Date"]
    Pathogen = row["Pathogen"]
    Breed = row["Breed"]
    Country = row["Country"]
    Province = row["Province"]
    SampleType = row["SampleType"]
    SampleResult = row["Result"]
    DiagnosticTest = row["DiagnosticTest"]
    PigStage= row["PigStage"]

    
    # Create a unique URI for each sample based on the row index
    PigSample_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#Lab6PigSample_{index}")
    
    # Add sample type assertion
    g.add((PigSample_uri, RDF.type, LHO.PigSample))
    description = "An individual representing a sample from a Pig and Piglets."
    g.add((PigSample_uri, RDFS.comment, Literal(description, lang="en")))
    
    if "Pathogen" in row and row["Pathogen"]:
        Pathogen = row["Pathogen"]
        Pathogen_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Pathogen}")
        g.add((PigSample_uri, LHO.hasPathogen, Pathogen_uri))
        g.add((Pathogen_uri, RDF.type, decide.Pathogen))
        description = "An individual representing Pig pathogen."
        g.add((Pathogen_uri, RDFS.comment, Literal(description, lang="en")))
        
    if "Breed" in row and row["Breed"]:
        Breed = row["Breed"]
        Breed_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Breed}")
        g.add((PigSample_uri, LHO.hasBreed, Breed_uri))
        g.add((Breed_uri, RDF.type, LHO.Breed))
        description = "An individual representing a Piglet breed which is Meat."
        g.add((Breed_uri, RDFS.comment, Literal(description, lang="en")))
        
    if "DiagnosticTest" in row and row["DiagnosticTest"]:
        DiagnosticTest= row["DiagnosticTest"]
        DiagnosticTest_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{DiagnosticTest}")
        g.add((PigSample_uri, LHO.hasDiagnosticTest, DiagnosticTest_uri))
        g.add((DiagnosticTest_uri, RDF.type, LHO.DiagnosticTest))
    if DiagnosticTest == "PCR":
        description = "An individual representing DNA/RNA amplification for rapid pathogen detection."
    elif SampleType == "Culture":
        description = "An individual representing a Growing live microorganisms for identification."
        g.add((DiagnosticTest_uri, RDFS.comment, Literal(description, lang="en")))
        
    if "Country" in row and row["Country"]:
        Country = row["Country"].strip()  # Use strip() to remove leading and trailing spaces
        # Encode the Country value to create a valid URI
        Country = urllib.parse.quote(Country)
        Country_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Country}")
    
        # Map to the specific class URI in your ontology and add the label
        g.add((PigSample_uri, LHO.hasCountry, Country_uri))
        g.add((Country_uri, RDF.type, URIRef("http://purl.obolibrary.org/obo/NCIT_C25464")))  # Use the specific class URI
        g.add((Country_uri, RDFS.label, Literal("Spain")))
        description = "An individual representing different Countries."
        g.add((Country_uri, RDFS.comment, Literal(description, lang="en")))
        
    if "Province" in row and row["Province"]:
        Province = row["Province"].strip()
        Province = urllib.parse.quote(Province)
        Province_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Province}")
        g.add((PigSample_uri, LHO.hasProvince, Province_uri))
        g.add((Province_uri, RDF.type, LHO.Province))
        description = "An individual representing different Province."
        g.add((Province_uri, RDFS.comment, Literal(description, lang="en")))
    
    if "PigStage" in row and row["PigStage"]:
        Stage = row["PigStage"]
        Stage_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Stage}")
        g.add((PigSample_uri, LHO.hasProductionStages, Stage_uri))
        g.add((Stage_uri, RDF.type, LHO.PigProductionStage))
        description = "An individual representing Production stage ."
        g.add((Stage_uri, RDFS.comment, Literal(description, lang="en")))
    
    if "SampleType" in row and row["SampleType"]:
        SampleType = row["SampleType"]
        SampleType_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{SampleType}")
        # Add statements for SampleType
        g.add((PigSample_uri, LHO.hasSampleType, SampleType_uri))
        g.add((SampleType_uri, RDF.type, URIRef("http://www.purl.org/decide#SampleType")))
    if SampleType == "Autopsy":
        description = "An individual representing a cattle sample obtained through autopsy."
    elif SampleType == "BAL":
        description = "An individual representing a cattle sample obtained through bronchoalveolar lavage (BAL)."
    elif SampleType == "SWAB":
        description = "An individual representing a cattle sample obtained through swabbing."
    elif SampleType == "Missing":
        description = "An individual representing a missing or unspecified cattle sample type."
        g.add((SampleType_uri, RDFS.comment, Literal(description, lang="en")))
           
    if "Result" in row and row["Result"]:
        SampleResult = row["Result"]
        # Create a unique URI for the Result based on the value
        Result_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{SampleResult}")
        g.add((PigSample_uri, LHO.hasResult, Result_uri))
        g.add((Result_uri, RDF.type, decide.SampleResult))
    if SampleResult == "1":
        description = "An individual representing a Positive test result"
    elif SampleResult == "0":
        description = "An individual representing a negative test result"
    elif SampleResult == "missing":
        description = "An individual representing an unknown or missing test result."
    g.add((Result_uri, RDFS.comment, Literal(description, lang="en")))


    if "Date" in row and not pd.isnull(row["Date"]):  # Check for NaN or None values
        Date = str(row["Date"]).strip()
    
    # Split the datetime string and take the date part
        Date = Date.split()[0]
    
    # Create a unique URI for the Date based on the value (use the Date directly)
        Date_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Date}")
    
    # Add Date assertion
        g.add((PigSample_uri, LHO.hasDate, Date_uri))
        g.add((Date_uri, RDF.type, LHO.Date))
    

# Serialize the RDF graph to a file
rdf_output_file = "output/PigLab6.rdf"
g.serialize(rdf_output_file, format="xml")

# Serialize the RDF graph to Turtle format and print it too see RDF conversion
turtle_data = g.serialize(format="turtle")
print(turtle_data)

### Step 4: For Knowledge Graph,Load the RDF data and ontology into a Panda DataFrame: 

In [ ]:
# Parse the ontology file in OWL format and add it to the graph
path_to_ontology = "Ontology/LivestockHealthOntology.owl"
g.parse(path_to_ontology, format="xml")

### Step 05: Query the data from updated ontology 

In [ ]:

query = """
PREFIX decide: <http://www.purl.org/decide#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX LHO: <http://www.purl.org/decide/LiveStockHealthOnto/LHO#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT
  (strafter(str(?Sample), "#") AS ?SampleName)
  (strafter(str(?Pathogen), "#") AS ?PathogenName)
  (strafter(str(?Breed), "#") AS ?BreedName)
  (strafter(str(?SampleType), "#") AS ?SampleTypeName)
  (strafter(str(?DiagnosticTest), "#") AS ?DiagnosticTestName)
  (strafter(str(?PigProductionStages), "#") AS ?ProductionStage)
  (strafter(str(?Province), "#") AS ?ProvinceName)
  (strafter(str(?Country), "#") AS ?CountryName)
  (strafter(str(?SampleResult), "#") AS ?SampleResultName)

WHERE {
 {
  ?Sample rdf:type LHO:PigSample .
  ?Sample LHO:hasPathogen ?Pathogen .
  ?Sample LHO:hasBreed ?Breed .
  ?Sample LHO:hasSampleType ?SampleType .
  ?Sample LHO:hasProvince ?Province .
  ?Sample LHO:hasCountry ?Country .
  ?Sample LHO:hasProductionStages ?PigProductionStages .
  ?Sample LHO:hasResult ?SampleResult .
  ?Sample LHO:hasDiagnosticTest ?DiagnosticTest .
  }
}
"""


# Execute the query and retrieve the results
results = g.query(query)

# Convert the results to a Pandas dataframe
data = []
for row in results:
    data.append(list(row))

df = pd.DataFrame(data, columns=[
    "Sample", "Pathogen", "Breed", "SampleType", 
    "DiagnosticTest","PigProductionStage", "Province", "Country", "SampleResult"
])

# Display the dataframe
df
